# Normalizing Clinical Samples Using scMerge 

In order to determine if batch effects exist, the bridging controls were used to normalize the data since they are approximately equally-distributed across both batches and are designed to serve as controls for the studies. 

Using the bridge controls, we can then use that "information" that was learned and apply that to the clinical samples and determine how effective normalization is. 

The analyses below use multi-variate ANOVAs and graphical visualizations (i.e., PCA, T-SNE, and UMAP) to compare scMerge's normalization and batch correction from the leukopak bridging controls to the clinical samples.


In [8]:
## Load data
options(warn=-1)

# sce<-readRDS("../../data/sceFltrd_kpGns.rds")
# sce<-sce[,sce$subjid!="IMM19"]
# sce$subjid<-factor(sce$subjid,levels=sort(unique(sce$subjid)))
# subSmpl.lst<-readRDS("../../data/subSmpl.lst.b15p10.rds")
# print('data loaded')
# fres <- readRDS('../../data/tmp_scMerge.RDS')

In [2]:
## Load libraries 
options(warn=-1)
source('../code/scMerge_helperFunctions.R')
library(SingleCellExperiment)
require(scMerge)
require(BiocParallel)
require(ggpubr)

Loading required package: scMerge

Loading required package: BiocParallel

Loading required package: ggpubr

Loading required package: ggplot2



In [35]:
length(subSmpl.lst) # length(subSmpl.lst): 6  ; dim(subSmpl.lst[[1]]) (bootstrap x cells: 15 x 7353)
wks<-sort(unique(sce$wk))
names(fres)

#### Extract Elements from scMerge/RUV normalization
require(scMerge)
data("segList_ensemblGeneID", package = "scMerge")
seg_index <- segList_ensemblGeneID$human$human_scSEG
cmSEGs <- intersect(row.names(fres$sce_object), seg_index)


k<-fres[[2]]$optimal_ruvK ## k=1
falpha<-fres[[2]][[k]]$fullalpha # dim(falpha) 52 x 20240 (k x G)
colnames(falpha)<-rownames(fres$sce_object)
alpha <- falpha[seq_len(min(k, nrow(falpha))), , drop = FALSE] # dim(alpha) : 1 x 20240
ac <- alpha[,cmSEGs, drop = FALSE] # dim(ac): 1 x 52; sum(is.na(ac[1,]))
head(ac)
d=1;b=1
tmp <- fres$sce_object

dim(sce)
dim(tmp)

[1] 6

[1] "sce_object"      "scNormalization"

ENSG00000131043,ENSG00000161204,ENSG00000140526,ENSG00000146109,ENSG00000114331,ENSG00000112304,ENSG00000143727,ENSG00000138071,ENSG00000137845,ENSG00000159346,⋯,ENSG00000182986,ENSG00000198521,ENSG00000204604,ENSG00000177853,ENSG00000075292,ENSG00000172687,ENSG00000267041,ENSG00000106400,ENSG00000132485,ENSG00000214941
2.495481,6.605822,9.757801,6.996716,11.51579,8.361165,12.06898,18.14815,12.18384,8.616206,⋯,2.882332,4.874293,4.422323,7.954154,11.78824,5.688845,3.08727,9.864748,14.9074,8.576998


[1]  22758 496865

[1] 19524  9699

In [42]:
# for(d in 1: length(subSmpl.lst)){
#   for(b in 1:nrow(subSmpl.lst[[d]])){
    idx <- subSmpl.lst[[d]][b, ]
    sub.sce <- sce[which(rownames(sce) %in% rownames(tmp)) , which(sce$wk==wks[d])[idx] ]
#     all.equal(rownames(sub.sce),rownames(tmp)); unique(sub.sce$wk)==wks[d]
    
    scale_res <- standardize2_mod(Y=assay(sub.sce,"logcpm"), batch=sub.sce$batch)
    stand_tY <- DelayedArray::t(scale_res$stand_Y)
    stand_sd <- sqrt(scale_res$stand_var)
    stand_mean <- scale_res$stand_mean
    
    W <- stand_tY[, cmSEGs] %*% DelayedArray::t(ac) %*% solve(ac %*% DelayedArray::t(ac))
    newY_mc <- stand_tY - W %*% alpha
    ## Add back the mean and sd to the normalised data
    newY_mc <-(t(newY_mc) * stand_sd) + stand_mean
    assay(sub.sce,"normalized")<-newY_mc

In [43]:
sub.sce

class: SingleCellExperiment 
dim: 19524 7353 
metadata(0):
assays(3): counts logcpm normalized
rownames(19524): ENSG00000131043 ENSG00000161204 ... ENSG00000205439
  ENSG00000184274
rowData names(19): geneid genename ... Dendriticcell.exprProp
  Platelets.exprProp
colnames(7353): 5475 1007 ... 377997 386355
colData names(21): barcodes batch ... mito propMito
reducedDimNames(0):
altExpNames(0):

In [ ]:
#     ##################### LP: Run PCA - PRIOR/POST #####################################
#     BC.prior.smpl = scater::runPCA(sub.sce, exprs_values = "logcpm",ncomponents=5); 
#     #reducedDimNames(sceLP); head(reducedDim(sceLP)); dim(reducedDim(sceLP))
#     PCs.prior.smpl<-data.frame(cbind(reducedDim(BC.prior.smpl),"cellType"=colData(sub.sce)$cellType,"batch"=colData(sub.sce)$batch))
#     PCs.prior.smpl[,1:5]<-apply(PCs.prior.smpl[,1:5],2,function(x){as.numeric(as.character(x))})
#     op <- options(contrasts = c("contr.helmert", "contr.poly"))
#     maov.prior.smpl<-manova(cbind(PC1,PC2,PC3,PC4,PC5) ~ as.factor(cellType)*as.factor(batch),data=PCs.prior.smpl)
#     summary(maov.prior.smpl)
    
#     p<-scater::plotPCA(BC.prior.smpl, colour_by = "cellType", shape_by = "batch")
#     p<-scater::plotPCA(BC.prior.smpl, colour_by = "batch")

    
#     BC.post.smpl = scater::runPCA(sub.sce, exprs_values = "normalized",ncomponents=5); 
#     PCs.post.smpl<-data.frame(cbind(reducedDim(BC.post.smpl),"cellType"=colData(sub.sce)$cellType,"batch"=colData(sub.sce)$batch))
#     PCs.post.smpl[,1:5]<-apply(PCs.post.smpl[,1:5],2,function(x){as.numeric(as.character(x))})
#     op <- options(contrasts = c("contr.helmert", "contr.poly"))
#     maov.post.smpl<-manova(cbind(PC1,PC2,PC3,PC4,PC5) ~ as.factor(cellType)*as.factor(batch),data=PCs.post.smpl)
#     summary(maov.post.smpl)
    
#     p<-scater::plotPCA(BC.post.smpl, colour_by = "cellType", shape_by = "batch")
#     p<-scater::plotPCA(BC.post.smpl, colour_by = "batch")
    
#     ##################### LP: Run TSNE - PRIOR/POST #####################################
#     set.seed(1000)
#     BC.prior.smpl <- scater::runTSNE(BC.prior.smpl, perplexity=50, dimred="PCA", exprs_values = "logcpm",  n_dimred=5)
    
#     p<-scater::plotTSNE(BC.prior.smpl, colour_by = "cellType", shape_by = "batch")
#     p<-scater::plotTSNE(BC.prior.smpl, colour_by = "batch")
    
#     BC.post.smpl <- scater::runTSNE(BC.post.smpl, perplexity=50, dimred="PCA", exprs_values = "normalized",  n_dimred=5)
#     p<-scater::plotTSNE(BC.post.smpl, colour_by = "cellType", shape_by = "batch")
#     p<-scater::plotTSNE(BC.post.smpl, colour_by = "batch")
    
#     ##################### LP: Run UMAP - PRIOR/POST #####################################
#     BC.prior.smpl <- scater::runUMAP(BC.prior.smpl,  exprs_values = "logcpm",  n_dimred=5)
    
#     outfile = "BCPrior.umap1.smpl.pdf" 
#     p<-scater::plotUMAP(BC.prior.smpl, colour_by = "cellType", shape_by = "batch")
#     pdf(file = outfile, width = 7, height = 5)
#     print(p)
#     dev.off()
#     outfile = "BCPrior.umap2.smpl.pdf" 
#     p<-scater::plotUMAP(BC.prior.smpl, colour_by = "batch")
#     pdf(file = outfile, width = 7, height = 5)
#     print(p)
#     dev.off()
    
#     BC.post.smpl <- scater::runUMAP(BC.post.smpl, exprs_values = "normalized", use_dimred="PCA",  n_dimred=5) 
    
#     outfile = "BCPost.umap1.smpl.pdf" 
#     p<-scater::plotUMAP(BC.post.smpl, colour_by = "cellType", shape_by = "batch")
#     pdf(file = outfile, width = 7, height = 5)
#     print(p)
#     dev.off()
#     outfile = "BCPost.umap2.smpl.pdf" 
#     p<-scater::plotUMAP(BC.post.smpl, colour_by = "batch")
#     pdf(file = outfile, width = 7, height = 5)
#     print(p)
#     dev.off()
#   }
# }